In [1]:
import numpy as np
import pandas as pd
import speech_recognition as sr
import wave, os, glob
import librosa as lb
from pydub import AudioSegment
import IPython.display as ipd
import math as m

import tensorflow as tf
tf.reset_default_graph()
from sklearn.metrics import confusion_matrix
from sklearn import datasets
from sklearn.model_selection import train_test_split

from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
%matplotlib inline

C:\Users\Kishore Thamilvanan\Anaconda3\lib\site-packages\pydub\utils.py:165: RuntimeWarning: Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work
  warn("Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work", RuntimeWarning)


## Preparation

In [2]:
filepath = 'data/Data/'
files = os.listdir(filepath)
# print(files)

In [3]:
speeches = []
# iterate over all the files in directory 'parent'
for each_celeb in os.listdir(filepath):
    celeb_speech = []
    current_path = "".join((filepath, each_celeb))
    
    if not os.path.isdir(current_path):
        break    
    
#     celeb_speech.append(each_celeb)
    
    for each_speech in os.listdir(current_path):
        next_path = "".join((current_path, "/", each_speech))
        
        
        for each_wav in os.listdir(next_path):
            
            p = "".join((next_path, "/", each_wav))
            
#             samplerate, d = wavfile.read(p)
            wave, sr = lb.load(p)
            celeb_speech.append(wave)
    
    celeb_speech = np.concatenate(celeb_speech)
    speeches.append(celeb_speech)


## Feature extraction

In [4]:
spectrograms=[]

while len(spectrograms)<10:
    spec = lb.feature.melspectrogram(y=speeches[0], sr=8000, hop_length = int(8000*0.01), n_fft = int(8000*0.03))
    spectrograms.append(spec)
  

In [5]:
A = np.asarray(spectrograms)

In [19]:
A.shape

(10, 128, 251042)

In [20]:
A.shape = (A.shape[2]*A.shape[0], A.shape[1])

In [21]:
A.shape

(2510420, 128)

## Labels

In [24]:
y = []
# print(y.shape)

In [25]:
for i in range(0,2510421):
    x = []
    for j in range (1, 129):
        x.append(i+1)
    y.append(x)

In [ ]:
y = np.asarray(y)

## Training && Confusion matrix

We are building a classifier using a feedfoward neural network with one hidden layer and 256 hidden units.

This model is trained for 1000 epochs. Please run the program while grading.

In [ ]:
RANDOM_SEED = 1
tf.set_random_seed(RANDOM_SEED)


def init_weights(shape):
    """ Weight initialization """
    weights = tf.random_normal(shape, stddev=0.1)
    return tf.Variable(weights)

def forwardprop(X, w_1, w_2):
    """
    Forward-propagation.
    IMPORTANT: yhat is not softmax since TensorFlow's softmax_cross_entropy_with_logits() does that internally.
    """
    h    = tf.nn.relu(tf.matmul(X, w_1))  # The \sigma function
    yhat = tf.matmul(h, w_2)  # The \varphi function
    return yhat

# This x_ddta is A and y_data is y in our assignment.
def get_data():
    return train_test_split(A, y, test_size=0.33, random_state=RANDOM_SEED)

def main():
    train_X, test_X, train_y, test_y = get_data()

    # HINT: Turn the targets to categorical
    # Use tf.keras.utils.to_categorical to convert the target variables
    train_y = tf.keras.utils.to_categorical(train_y)
    test_y = tf.keras.utils.to_categorical(test_y)
    
    # Layer's sizes
    x_size = train_X.shape[1]   
    # TODO: Choose appropriate hidden sizes for both layers
    h_size = 256 
    y_size = train_y.shape[1]


    # Symbols
    X = tf.placeholder("float", shape=[None, x_size])
    y = tf.placeholder("float", shape=[None, y_size])

    # Weight initializations
    w_1 = init_weights((x_size, h_size))
    w_2 = init_weights((h_size, y_size))
    
    # Bias Section
    # TODO : You can create three bias variables and use forwardprop_with_bias function
    
    # Forward propagation
    # TODO: create `yhat` using forwardprop/forwardprop_with_bias function
    yhat    = forwardprop(X,w_1,w_2)
    predict = tf.argmax(yhat, axis=1)

    # Backward propagation
    
    # Create cost function and Optimizer 
    # Loss Function: Cross Entropy loss
    # Optimizer : Gradient Descent/Adam Opt.
    cost    = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y, logits=yhat))
    updates = tf.train.AdamOptimizer().minimize(cost)

    # Run SGD
    sess = tf.Session()
    init = tf.global_variables_initializer()
    sess.run(init)

    # Epoch count
    epoc_count = 1000
    
    for epoch in range(epoc_count):
        # Train with each example
        for i in range(len(train_X)):
            sess.run(updates, feed_dict={X: train_X[i: i + 1], y: train_y[i: i + 1]})
        #TODO: Print Test and Train accuracy
        
    sess.close()

if __name__ == '__main__':
    main()

## K-means Clustering

Now we use the K-means algorithm cluster A into 10 clusters.

In [ ]:
kmeans = KMeans(n_clusters = 10)
kmeans.fit(A)

# Plot the Models Classifications
plt.scatter(A[:,0], y[:, 0], c=kmeans.labels_, cmap='rainbow')